In [ ]:
import json
from rdflib import Graph, URIRef, Literal, Namespace

# Load JSON data
json_data = '{"title": "No-Bake Nut Cookies", "link": "www.cookbooks.com/Recipe-Details.aspx?id=44874", "ingredientSet": [{"name": "brown", "oboId": "PATO_0000952", "match": "Full Term Match"}, {"name": "sugar", "oboId": "FOODON_03420108", "match": "Full Term Match"}, {"name": "milk", "oboId": "UBERON_0001913", "match": "Full Term Match"}, {"name": "vanilla", "oboId": "NCBITAXON_51238", "match": "Full Term Match"}, {"name": "nut (whole or parts)", "oboId": "FOODON_03306632", "match": "Full Term Match"}, {"name": "carya illinoinensis", "oboId": "NCBITAXON_32201", "match": "Full Term Match"}, {"name": "butter", "oboId": "FOODON_03310351", "match": "Full Term Match"}, {"name": "margarine", "oboId": "FOODON_03301672", "match": "Full Term Match"}, {"name": "divided into pieces, thickness <0.3 cm.", "oboId": "FOODON_03430100", "match": "Full Term Match"}, {"name": ["rice", "scone"], "oboId": ["FOODON_00004327", "FOODON_00002465"], "match": "Component Match"}]}'

# Parse JSON data to Python object
data = json.loads(json_data)

# Create RDF graph
graph = Graph()

# Define namespaces
FOOD = Namespace('http://example.com/food')
graph.bind('food', FOOD)

# Create recipe resource
recipe_uri = URIRef(FOOD + 'recipe/1')
graph.add((recipe_uri, RDF.type, URIRef('http://schema.org/Recipe')))
graph.add((recipe_uri, URIRef('http://schema.org/name'), Literal(data['title'])))
graph.add((recipe_uri, URIRef('http://schema.org/url'), Literal(data['link'])))


# Add ingredient resources
for index, ingredient in enumerate(data['ingredientSet']):
    if isinstance(ingredient['oboId'], list):
        op_alternative_uri = URIRef(f"{FOOD}/ingredient/{index}")
        
        for ingredient_name, ingredient_id in zip(ingredient['name'], ingredient['oboId']):
            
            ingredient_uri = URIRef(f"{FOOD}/ingredient/{index}/{ingredient_id}")
            graph.add((ingredient_uri, RDF.type, URIRef('http://schema.org/Text')))
            graph.add((ingredient_uri, URIRef('http://schema.org/name'), Literal(ingredient_name)))
            graph.add((op_alternative_uri, URIRef('http://schema.org/recipeIngredient'), ingredient_uri))
        
        # attach "or" into recipie as ingredient
        graph.add((recipe_uri, URIRef('http://schema.org/operators/or'), op_alternative_uri))
    else:
        ingredient_uri = URIRef(f"{FOOD}/ingredient/{index}/{ingredient['oboId']}")
        graph.add((recipe_uri, URIRef('http://schema.org/recipeIngredient'), ingredient_uri))
        graph.add((ingredient_uri, RDF.type, URIRef('http://schema.org/Text')))
        graph.add((ingredient_uri, URIRef('http://schema.org/name'), Literal(ingredient['name'])))
        

# Print the graph in Turtle format
print(graph.serialize(format='xml'))